# Multi-Omics Imputation

The plan is to do the following:
- Divide the data into train, validation, and test sets. Leave the test set for later.
- Remove the values for a certain omics type from the validation set
- Impute using the train set and the remaining value in the validation set
- Compare these imputed values against the true values
    - Distribution of correlation coefficients
    - Get the mean and stdev of the correlation coefficients
    - Choose best model
- Evaluate on test set
- Choose best method
- Try on independent set
- Finally, train GCN model and see difference between single omics, multi-omics, and imputed multi-omics

I'll first start with some basics: data import and processing.
Then I'll move to imputing one omics from two.
Then I'll move to imputing two omics from one.
I'll do all the steps above along the way.

## Importing requisite packages

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

## Importing data

In [7]:
mrna = pd.read_csv("../R/TCGA BRCA/mrna_top1000.csv", index_col=0)
meth = pd.read_csv("../R/TCGA BRCA/meth_top1000.csv", index_col=0)
mirna = pd.read_csv("../R/TCGA BRCA/mirna_anova.csv", index_col=0)

labels = pd.read_csv("../R/TCGA BRCA/PAM50_subtype.csv", index_col=0)

## Basic Data Processing

Just combining all data and then also having a list containing what datatype the columns belong to.

In [3]:
all_data = pd.merge(pd.merge(mrna, meth, left_index=True, right_index=True), mirna,  left_index=True, right_index=True)

datatypes = ["mrna"]*mrna.shape[1] + ["meth"]*meth.shape[1] + ["mirna"]*mirna.shape[1]

In [4]:
all_data.head()

,DBF4|10926,DACH1|1602,BBS4|585,L3MBTL4|91133,TK1|7083,KIAA1370|56204,GPD1L|23171,RERG|85004,RAPGEF3|10411,FBXO36|130888,...,hsa-mir-217,hsa-mir-424,hsa-mir-581,hsa-mir-483,hsa-mir-3614,hsa-mir-16-1,hsa-mir-550a-2,hsa-mir-24-1,hsa-mir-508,hsa-mir-642a
patient_id,,,,,,,,,,,,,,,,,,,,,
TCGA-D8-A1XU-01,6.998086,10.125705,8.332896,4.537035,8.211207,10.111945,10.140800,12.116795,8.603795,8.009364,...,0.004411,0.078464,0.3125,0.002911,0.001617,0.059937,0.046753,0.256461,0.010449,0.070690
TCGA-D8-A1XV-01,8.117341,11.005011,9.888690,7.498527,9.495441,12.370753,10.892996,11.942934,7.569660,8.433369,...,0.003170,0.012424,0.1250,0.000333,0.002426,0.082900,0.038961,0.037773,0.003380,0.032759
TCGA-E9-A1N3-01,8.100588,13.055740,9.338570,2.633733,10.136103,10.892182,10.167938,11.853222,8.277645,8.966269,...,0.003446,0.044661,0.6875,0.000832,0.002426,0.063510,0.049351,0.093439,0.053473,0.110345
TCGA-C8-A1HE-01,7.399941,11.590931,9.617624,3.854843,9.238226,11.675139,12.240305,11.507753,9.101567,8.740199,...,0.001792,0.057869,0.1250,0.001248,0.004448,0.066019,0.007792,0.041750,0.001537,0.044828
TCGA-A1-A0SQ-01,7.721054,9.337846,9.063252,3.938314,9.899231,10.615200,10.953656,11.271823,7.966302,7.418308,...,0.001103,0.005261,0.0625,0.000083,0.003235,0.010478,0.005195,0.009940,0.000615,0.155172


In [4]:
labels.head()

,cancer_subtype
patient_id,
TCGA-D8-A1XU-01,LumA
TCGA-D8-A1XV-01,LumA
TCGA-E9-A1N3-01,LumA
TCGA-C8-A1HE-01,LumA
TCGA-A1-A0SQ-01,LumA


Doing the train-validation-test split.
These contain all the values intact.  

Here, we do a 60-20-20 split.

In [9]:
X_train, X_test, y_train, y_test = train_test_split(all_data, labels, test_size = 0.2, random_state = 42, stratify = labels)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.25, random_state = 42, stratify = y_train)


print(X_train.shape)
print(X_val.shape)
print(X_test.shape)
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

#all_data.head()
#labels.head()
#X_train.head()
#y_train.head()

(372, 2257)
(125, 2257)
(125, 2257)
(372, 1)
(125, 1)
(125, 1)


Removing all miRNA feature values

In [10]:
#Keeping values for later
from copy import deepcopy
X_test_truth = deepcopy(X_test)
X_val_truth = deepcopy(X_val)

mask = [x=="mirna" for x in datatypes]
X_test.loc[:,mask] = np.nan
X_val.loc[:,mask] = np.nan

X_test.loc[:,mask].head()

/home/bs16b001/.local/lib/python3.7/site-packages/pandas/core/indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/home/bs16b001/.local/lib/python3.7/site-packages/pandas/core/indexing.py:691: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


,hsa-mir-576,hsa-mir-200b,hsa-mir-3687,hsa-mir-126,hsa-mir-26a-2,hsa-mir-101-1,hsa-mir-218-2,hsa-mir-223,hsa-mir-335,hsa-mir-1468,...,hsa-mir-217,hsa-mir-424,hsa-mir-581,hsa-mir-483,hsa-mir-3614,hsa-mir-16-1,hsa-mir-550a-2,hsa-mir-24-1,hsa-mir-508,hsa-mir-642a
patient_id,,,,,,,,,,,,,,,,,,,,,
TCGA-A2-A3XX-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-BH-A0DI-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-A7-A6VX-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-BH-A0AZ-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-AR-A1AX-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Simple Imputation Methods

In [15]:
from sklearn.impute import SimpleImputer

#Combining the train and test samples into one dataframe.
print(X_train.shape)
print(X_val.shape)
X = pd.concat([X_train, X_val])
print(X.shape)
X.iloc[370:375, mask]

(372, 2257)
(125, 2257)
(497, 2257)


,hsa-mir-576,hsa-mir-200b,hsa-mir-3687,hsa-mir-126,hsa-mir-26a-2,hsa-mir-101-1,hsa-mir-218-2,hsa-mir-223,hsa-mir-335,hsa-mir-1468,...,hsa-mir-217,hsa-mir-424,hsa-mir-581,hsa-mir-483,hsa-mir-3614,hsa-mir-16-1,hsa-mir-550a-2,hsa-mir-24-1,hsa-mir-508,hsa-mir-642a
patient_id,,,,,,,,,,,,,,,,,,,,,
TCGA-D8-A1JB-01,0.079457,0.046150,0.008850,0.152926,0.099120,0.103930,0.080421,0.056304,0.022416,0.021445,...,0.000965,0.101410,0.0000,0.007236,0.002022,0.022180,0.033766,0.055666,0.006146,0.005172
TCGA-D8-A1JL-01,0.110465,0.138854,0.017699,0.097901,0.139139,0.027866,0.097684,0.076584,0.062983,0.020316,...,0.115024,0.202149,0.5625,0.000250,0.020218,0.083382,0.033766,0.236581,0.003688,0.008621
TCGA-EW-A2FS-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-BH-A8FZ-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-D8-A1XG-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Imputing with Mean and Median

### Imputing with Mean

In [16]:
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(X_train)

mean_imputed = imp.transform(X_val)
# SimpleImputers returns a numpy.ndarray
# I will convert it to a pandas data frame
#mean_imputed = pd.DataFrame(mean_imputed, columns = X_val.columns, index = X_val.index)


#print(mean_imputed.shape)
#mask = [x=="mirna" for x in datatypes]
#mean_imputed.loc[:,mask].head()

### Imputing with Median

In [17]:
imp = SimpleImputer(missing_values=np.nan, strategy='median')
imp.fit(X_train)

median_imputed = imp.transform(X_val)
#median_imputed = pd.DataFrame(median_imputed, columns = X_val.columns, index = X_val.index)

#print(median_imputed.shape)
#mask = [x=="mirna" for x in datatypes]
#median_imputed.loc[:,mask].head()

# Slightly More Complicated Methods

In [15]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

Here, initially, all the missing features are replaced with the mean value. Then, iteratively, the estimator is used to estimate the missing value from all the other features. Within each iteration through all the missing values, the features are imputed in a random order.

## Estimator: ElasticNet

#### L1 Ratio = 0.2

In [16]:
from sklearn.linear_model import ElasticNet

imp = IterativeImputer(estimator = ElasticNet(l1_ratio = 0.2), initial_strategy = "mean", imputation_order = "random", random_state = 42)
imp.fit(X_train)

elasticnet_l2_imputed = imp.transform(X_val)
#elasticnet_l2_imputed = pd.DataFrame(elasticnet_l2_imputed, columns = X_val.columns, index = X_val.index)
#print(elasticnet_l2_imputed.shape)
#mask = [x=="mirna" for x in datatypes]
#elasticnet_l2_imputed.loc[:,mask].head()

KeyboardInterrupt: 

#### L1 Ratio = 0.8

In [ ]:
from sklearn.linear_model import ElasticNet

imp = IterativeImputer(estimator = ElasticNet(l1_ratio = 0.8), initial_strategy = "mean", imputation_order = "random", random_state = 42)
imp.fit(X_train)

elasticnet_l1_imputed = imp.transform(X_val)
#elasticnet_l1_imputed = pd.DataFrame(elasticnet_l1_imputed, columns = X_val.columns, index = X_val.index)
#print(elasticnet_l1_imputed.shape)
#mask = [x=="mirna" for x in datatypes]
#elasticnet_l1_imputed.loc[:,mask].head()

## Estimator: KNeighborsRegressor

#### K = 15

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

imp = IterativeImputer(estimator = KNeighborsRegressor(n_neighbors=15), 
                       initial_strategy = "mean", imputation_order = "random", random_state = 42)
imp.fit(X_train)

knn15_iter_imputed = imp.transform(X_val)
knn15_iter_imputed = pd.DataFrame(knn15_iter_imputed, columns = X_val.columns, index = X_val.index)


print(knn15_iter_imputed.shape)
mask = [x=="mirna" for x in datatypes]
knn15_iter_imputed.loc[:,mask].head()

## Estimator: RandomForestRegressor

#### Max Depth = 3

In [ ]:
from sklearn.ensemble import RandomForestRegressor

imp = IterativeImputer(estimator = RandomForestRegressor(max_depth=3), 
                       initial_strategy = "mean", imputation_order = "random", random_state = 42)
imp.fit(X_train)

rf3_imputed = imp.transform(X_val)
# SimpleImputers returns a numpy.ndarray
# I will convert it to a pandas data frame
rf3_imputed = pd.DataFrame(rf3_imputed, columns = X_val.columns, index = X_val.index)


print(rf3_imputed.shape)
mask = [x=="mirna" for x in datatypes]
rf3_imputed.loc[:,mask].head()

#### Max Depth = 7

In [ ]:
from sklearn.ensemble import RandomForestRegressor

imp = IterativeImputer(estimator = RandomForestRegressor(max_depth=7), 
                       initial_strategy = "mean", imputation_order = "random", random_state = 42)
imp.fit(X_train)

rf7_imputed = imp.transform(X_val)
rf7_imputed = pd.DataFrame(rf7_imputed, columns = X_val.columns, index = X_val.index)


print(rf7_imputed.shape)
mask = [x=="mirna" for x in datatypes]
rf7_imputed.loc[:,mask].head()

# Deck Imputation

It is not exactly deck imputation in that we are not replacing the missing value with a value from the existing set. Here, I select the k closest samples and get the average of their values to impute the missing values.

I am testing kNN with k values equal to all odd numbers between 0 and 20.

In [11]:
from sklearn.impute import KNNImputer

knn = {}

for i in [1,5,10,15,20,25,30,35,40,45,50,75,100]:
    imputer = KNNImputer(n_neighbors=i)
    imputer.fit(X_train)

    knn[i] = imputer.transform(X_val)
    #knn[i] = pd.DataFrame(knn[i], columns = X_val.columns, index = X_val.index)

print(len(knn))
#print(knn[1].shape)
#mask = [x=="mirna" for x in datatypes]
#knn[1].loc[:,mask].head()

13


# Comparing Imputation Methods

To compare the imputation methods, we first need to quantify them. Here, I am going to use the Normalized Root Mean Squared Error (NRMSE) to quantify each of the methods and then compare them.

< Insert NRMSE formula in latex >

In [19]:
from scipy.stats import iqr

def _error(actual: np.ndarray, predicted: np.ndarray):
    """ Simple error """
    return actual - predicted

def mse(actual: np.ndarray, predicted: np.ndarray):
    """ Mean Squared Error """
    return np.mean(np.square(_error(actual, predicted)))


def rmse(actual: np.ndarray, predicted: np.ndarray):
    """ Root Mean Squared Error """
    return np.sqrt(mse(actual, predicted))


def nrmse(actual: np.ndarray, predicted: np.ndarray):
    """ Normalized Root Mean Squared Error """
    return rmse(actual, predicted) / (iqr(actual))

In [11]:
a = np.random.rand(5,6)
b = np.random.rand(5,6)

print(_error(a,b))
print(mse(a,b))
print(rmse(a,b))
print(nrmse(a,b))

[[ 0.14393372  0.27895817  0.09184603 -0.48962765  0.61275826  0.46230607]
 [ 0.19124946 -0.39554615 -0.1028973  -0.29488289  0.86849221  0.53023171]
 [-0.09447638 -0.04069172 -0.38886861  0.16560578 -0.22812985  0.45319997]
 [-0.54753045 -0.7367489   0.01852775 -0.9560363   0.40134552  0.20277053]
 [-0.18642123 -0.65511136  0.018154    0.1459206  -0.27313436 -0.22732483]]
0.1759840640209514
0.4195045458883031
0.9520721234696103


In [20]:
# imputed_values = {
#     "mean" : mean_imputed,
#     "median" : median_imputed,
#     "enet_l1" : elasticnet_l1_imputed,
#     "enet_l2" : elasticnet_l2_imputed,
#     "knn5_iter" : knn5_iter_imputed,
#     "knn10_iter" : knn10_iter_imputed,
#     "knn15_iter" : knn15_iter_imputed,
#     "rf3" : rf3_imputed,
#     "rf7" : rf7_imputed,
#     "knn1" : knn[1],
#     "knn3" : knn[2],
#     "knn5" : knn[3],
#     "knn7" : knn[4],
#     "knn9" : knn[5],
#     "knn11" : knn[6],
#     "knn13" : knn[7],
#     "knn15" : knn[8],
#     "knn17" : knn[9],
#     "knn19" : knn[10]
# }

mask = [x=="mirna" for x in datatypes]
truth = X_val_truth.loc[:,mask].to_numpy()
random = (np.random.rand(truth.shape[0],truth.shape[1]))# - np.mean(truth))/np.std(truth)


print(nrmse(truth, truth))
print(nrmse(truth, random))
print(nrmse(truth, mean_imputed[:,mask]))
print(nrmse(truth, median_imputed[:,mask]))
#print(nrmse(truth, elasticnet_l1_imputed[:,mask]))
#print(nrmse(truth, elasticnet_l2_imputed[:,mask]))
#print(nrmse(truth, knn5_iter_imputed[:,mask]))
#print(nrmse(truth, knn10_iter_imputed[:,mask]))
#print(nrmse(truth, knn15_iter_imputed[:,mask]))
#print(nrmse(truth, rf3_imputed[:,mask]))
#print(nrmse(truth, rf5_imputed[:,mask]))
print(nrmse(truth, knn[1][:,mask]))
print(nrmse(truth, knn[5][:,mask]))
print(nrmse(truth, knn[10][:,mask]))
print(nrmse(truth, knn[15][:,mask]))
print(nrmse(truth, knn[20][:,mask]))
print(nrmse(truth, knn[25][:,mask]))
print(nrmse(truth, knn[30][:,mask]))
print(nrmse(truth, knn[35][:,mask]))
print(nrmse(truth, knn[40][:,mask]))
print(nrmse(truth, knn[45][:,mask]))
print(nrmse(truth, knn[50][:,mask]))
print(nrmse(truth, knn[75][:,mask]))
print(nrmse(truth, knn[100][:,mask]))

0.0
7.267944970941607
1.3514235544024777
1.382975035923613
1.666845541029768
1.400642780234503
1.363293181600634
1.3519965422151523
1.3438070871661862
1.340271141343888
1.3391920909888835
1.338449260953644
1.3354163368415288
1.3329650963154451
1.3313104726543543
1.3340476449208134
1.3356092116386185
